In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from pandas import json_normalize
import ast
nltk.download('vader_lexicon')

In [ ]:
#Ruta del archivo JSON
data_list = []
file_path = 'australian_user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
userReviewsDf = pd.DataFrame(data_list)

In [ ]:
userReviewsExplodedDf=userReviewsDf.explode('reviews').reset_index(drop=True)

In [ ]:
userReviewsExplodedDf['review']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('review') if isinstance(x, dict) and 'review' in x else None)

In [ ]:
def analyze_sentiment(text):
    if text is None:
        return '1'

    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(text)['compound']

    if text:
        if sentiment_score >= 0.05:
            return '2'
        elif sentiment_score <= -0.05:
            return '0'
        else:
            return '1'

In [ ]:
userReviewsExplodedDf['sentiment_analysis']=userReviewsExplodedDf['review'].apply(analyze_sentiment)

In [ ]:
del userReviewsExplodedDf['reviews']
del userReviewsExplodedDf['review']

In [ ]:
userReviewsExplodedDf.to_csv('userReviewsExploded.csv')
userReviewsExplodedDf.to_json('userReviewsExploded.json')